<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/pre_processingSyntheticResponseDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries

In [1]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.0 MB/s 
     |████████████████████████████████| 1.3 MB 67.9 MB/s 
     |████████████████████████████████| 365 kB 63.2 MB/s 
     |████████████████████████████████| 120 kB 58.0 MB/s 
     |████████████████████████████████| 6.6 MB 28.3 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 115 kB 61.8 MB/s 
     |████████████████████████████████| 127 kB 54.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import os
import pandas as pd 
import matplotlib.pyplot as plt

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict

import numpy as np
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [3]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cba403354b0a2d695562b21970acec19979656eae3c33abfb3fd52a68f25cf86
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Downloading a dataset

In [4]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

In [5]:
!ls

drive  emp_chatbot  sample_data


In [6]:
os.chdir('/content/emp_chatbot/data')

In [7]:
# https://drive.google.com/file/d/121M9d19m7ljXaw0f5pggpHd9gsN2Hu3U/view?usp=sharing
# !gdown --id 1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar
!gdown --id 1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
# https://drive.google.com/file/d/1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar/view?usp=sharing
# !gdown --id 1UhqyvObGgMl3_jEIkDiYJQEkQ7rFOXOc

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
To: /content/emp_chatbot/data/all_emotions_responses.json
100% 1.24M/1.24M [00:00<00:00, 161MB/s]


In [8]:
# data_files_EPru = {"train": "sadness_questions_follow_up.json"}
data_files_EPru = {"train": "all_emotions_responses.json"}
epru = load_dataset('json', data_files=data_files_EPru)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-48ab4b94264f4132/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

# Pre-computing Fluency and Embeddings




## Fluency/perplexity

In [10]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelWithLMHead, AutoTokenizer

device = "cuda"
# model_id = "sberbank-ai/ruT5-base"
model_id = "sberbank-ai/rugpt3large_based_on_gpt2"
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
model_LM = AutoModelWithLMHead.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
columns = ds.column_names
print(columns)

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?']


In [12]:
columns = ds.column_names
columns_indices = list(range(0,len(columns)))
perplexity_columns = [columns[index] for index in columns_indices if columns[index]!="emotion"]

In [13]:
perplexity_columns

['Have you strongly felt or expressed any of the following emotions towards someone?',
 'Do you believe you should be the savior of someone else?',
 'Do you feel that you are trying to control someone?',
 'Do you see yourself as the victim, blaming someone else for how negative you feel?',
 'Are you always blaming and accusing yourself for when something goes wrong?',
 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?',
 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?',
 'Was this emotion triggered by a specific event?',
 'Was the event recent?',
 'Did you find protocol 6 distressing?',
 'Did you find protocol 11 distressing?',
 'Is it ok to ask additional questions?',
 'Would you like to attempt protocols for joy?']

In [14]:
def calculate_perplexity(row, column_name):
  if row[column_name] != None and len(row[column_name]) != 0:
    encodings = tokenizer(row[column_name], return_tensors="pt")
    input_ids = encodings.input_ids.to(device)
    target_ids = input_ids.clone()
    with torch.no_grad():
      outputs = model_LM(input_ids, labels=target_ids)
      neg_log_likelihood = outputs[0]
    string_name = f"ppl_{column_name}"
    ppl = torch.exp(neg_log_likelihood)
    return {f'ppl_{column_name}': 1/ppl.item()}
  elif row[column_name] == None:
    return {f'ppl_{column_name}': 0}

In [15]:
for column_name in perplexity_columns:
  ds = ds.map(lambda x: calculate_perplexity(x, column_name))

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

...

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [16]:
# change dataset into pandas 
ds.set_format(type="pandas")
df = ds[:]
df.head()
# emotions = set(df['emotion'].tolist())

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,...,"ppl_Do you see yourself as the victim, blaming someone else for how negative you feel?",ppl_Are you always blaming and accusing yourself for when something goes wrong?,ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,ppl_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,ppl_Was this emotion triggered by a specific event?,ppl_Was the event recent?,ppl_Did you find protocol 6 distressing?,ppl_Did you find protocol 11 distressing?,ppl_Is it ok to ask additional questions?,ppl_Would you like to attempt protocols for joy?
0,"Я ценю то, что вы мне доверяете. Испытываете л...","Вы часто льстите себе, думая вы супер-герой д...","Спасибо вам за понимание. Скажите пожалуйста, ...",Благодарен вам за откровенность. Как вы счита...,"Благодарю вас за ответ. Вы часто считаете, что...",Благодарен вам за откровенность. вы часто наст...,Благодарю вас за честность и искренность. Еще...,Спасибо за открытость. Ваша тревога - это резу...,Спасибо за оказанное доверие. Это неприятное с...,"Благодарю за то, что поделились своими мыслями...",...,0.023105,0.023595,0.028663,0.020799,0.013067,0.016701,0.053717,0.005634,0.035500,0.0
1,Сейчас я хочу задать вам возможно не самый при...,"Мне кажетя, это не очень простой вопрос. Как ...","Мне очень понятно ваше состояние, и я прошу бе...","Я понимаю, вопрос может показаться слишком лич...","Спасибо вам за беседу, милый друг. Вы часто сч...","Знаю, следующий вопрос не будет легким. вы час...","Это замечательно, что вы такой открытый челове...",Я очень хочу вам помочь. Ваша тревога - это ре...,"Извините, если вам тяжело вспоминать об этом о...","Спасибо, что честно говорите о своих ощущениях...",...,0.026800,0.022207,0.027585,0.015447,0.018023,0.018230,0.047111,0.004214,0.043441,0.0
2,Испытываете ли к кому-нибудь негативные чувств...,Как часто вы ощущаете важность/обязательство ...,"Скажите пожалуйста, вы не замечали за собой чу...","Как вы считаете, хотя бы изредка вы позволяет...","Вы часто считаете, что в ваших проблемах тольк...","вы часто настаиваете на том, что только Ваша т...",Еще вопрос - какие у вас отношения с друзьями...,Ваша тревога - это результат определённого слу...,Это неприятное событие произошло недавно,"Мне жаль, что вы испытываете такие негативные ...",...,0.029281,0.016792,0.036355,0.015075,0.015587,0.003199,0.038144,0.000789,0.026303,0.0
3,"Вы часто льстите себе, думая вы супер-герой д...","Знаю, мои вопросы иногла задевабт за живое, я ...","Я благодарю вас за откровенность. Вам кажется,...","Я понимаю, что задаю вам сложный вопрос. Бывае...",Это конечно вопрос не простой. Часто ли вы вин...,"Спасибо, что доверились. Случается ли так, чт...","Спасибо, что открылись мне. Последний вопрос,...","Мне жаль, что вас охватило чувство тревоги. Вы...",Сочувствую вам. Оно произошло недавно?,"Спасибо, что наши в себе силы поделиться. Спас...",...,0.044692,0.051240,0.064644,0.015972,0.024866,0.013894,0.007949,0.015919,0.087401,0.0
4,"Мой милый друг, спасибо, что доверяете мне. У...","Спасибо за откровенность, для общей информации...","Это трудный вопрос, я понимаю. Вам кажется, чт...","Проанализируйте, пожалуйста, и откровенно отве...","Спасибо за честные ответы, это очень нам помож...","Я понимаю, возможно тяже

In [17]:
df[df['emotion']=='fear']['ppl_Did you find protocol 6 distressing?'].max()

0.05371661832265277

In [18]:
len(df.emotion.unique())
max_ppl = dict()
for emotion_type in df.emotion.unique():
  max_ppl.setdefault(emotion_type,[])
  for column in perplexity_columns:
    val = df[df['emotion']==emotion_type][f"ppl_{column}"].max()
    lst = max_ppl.get(emotion_type)
    lst.append(val)
    max_ppl[emotion_type] = lst

In [19]:
max_ppl['fear']


[0.11357946969976791,
 0.08428743654383698,
 0.10314861314764716,
 0.10416195438762227,
 0.16781518781415555,
 0.11877539156790956,
 0.09263736922932443,
 0.07410819705174645,
 0.04864111627561765,
 0.05371661832265277,
 0.038144073382641176,
 0.1996577217858484,
 0.0]

In [20]:
def normalise_ppl(row, column, largest_ppl):
  row[column] = row[column]/largest_ppl
  return row


In [21]:
ds.reset_format()

In [22]:
# ds1 = ds

In [23]:
# emotions = ['fear', 'sadness', 'anger', 'joy']
# mean_dict = dict()

# for emotion in emotions:
#   if emotion != "joy":
#     ds = ds1.filter(lambda row: row['emotion']==emotion)
#     mean = 0
#     for column_name in perplexity_columns[:-1]:
#       ds.set_format(type="pandas")
#       df = ds[:]
#       column_name = "ppl_"+column_name
#       df = df.replace(0, np.NaN)
#       mean += df[df[column_name] != 0][column_name].mean()
    
#     mean = mean/12
#     mean_dict[emotion] = mean
#   else:
#     ds = ds1.filter(lambda row: row['emotion']==emotion)
#     mean = 0
#     ds.set_format(type="pandas")
#     df = ds[:]
#     column_name = "ppl_"+perplexity_columns[-1]
#     df = df.replace(0, np.NaN)
#     mean += df[df[column_name] != 0][column_name].mean()

#     mean_dict[emotion] = mean

In [24]:
# mean_dict

In [25]:

normalised_ds = ds
for index, column in enumerate(perplexity_columns):
  normalised_ds = normalised_ds.map(lambda x: normalise_ppl(x, f"ppl_{column}", largest_ppl= max_ppl[x['emotion']][index]))

  0%|          | 0/600 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

...

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [26]:
# epru[train]["ppl_Did you find protocol 6 distressing?"]
normalised_ds["ppl_Have you strongly felt or expressed any of the following emotions towards someone?"]

[0.35592950660452455,
 0.3251960103858804,
 0.24321837865566898,
 0.15547863099316048,
 0.2699510958233901,
 0.3857066592734715,
 0.362266878342705,
 0.1259959615089407,
 0.21626028467743522,
 0.2970945775432052,
 0.2457352105700021,
 0.21626028467743522,
 0.4433741148333401,
 0.41978330264713687,
 0.3078559173812497,
 0.5368381798899562,
 0.15851773028748775,
 0.21626028467743522,
 0.1272419277425134,
 0.12076091761068193,
 0.09249066366446321,
 0.15664950575124711,
 0.2311327486039539,
 0.3290939731552935,
 0.3873600606343428,
 0.21091352986905487,
 0.3929967494653389,
 0.23793766878021547,
 0.42027362310986854,
 0.8393208295884586,
 0.09031908502503198,
 0.08593251497195493,
 0.07378229406071814,
 0.25865829950430663,
 0.3747918813214044,
 0.29669004256071835,
 0.3400662371187185,
 0.3979537224729476,
 0.8682274669591074,
 0.2893565722315835,
 0.3027997814562807,
 0.46643042575705274,
 0.2799387476284885,
 0.2784365447499395,
 0.2843737020495281,
 0.17687452385931807,
 0.20399507782

In [27]:
ds.reset_format()

## Empathy

In [28]:
# model_id = "mmillet/distilrubert-tiny-cased-conversational-v1_single_finetuned_empathy_classifier"
model_id = "mmillet/distilrubert-tiny-cased-conversational-v1_empathy_preprocessed_punct_lowercasing"
classifier = pipeline('text-classification', model=model_id)

In [29]:
preds = classifier(ds[8]["Have you strongly felt or expressed any of the following emotions towards someone?"], top_k=3)
preds = classifier("Это очень хорошо, что вы открыты, очень постараюсь вам помочь. Уверен, что у нас все получится. Задам вам дополнительные вопросы.", top_k=3)
print(preds)
prediction = max(preds, key=lambda x: x['score'])
print(prediction)

[{'label': 'LABEL_0', 'score': 0.8899251222610474}, {'label': 'LABEL_2', 'score': 0.09661342203617096}, {'label': 'LABEL_1', 'score': 0.013461414724588394}]
{'label': 'LABEL_0', 'score': 0.8899251222610474}


In [30]:
ds[8]["Have you strongly felt or expressed any of the following emotions towards someone?"]

'Вы чувствуете неприязнь к кому-либо'

In [31]:
# punctuation = ')£;-(:,?!'
# text_no_punct = ''.join([element for element in row['utterance'] if element not in punctuation])

def calculate_empathy(row, column_name):
  if row[column_name]!= None: #len(row[column_name]) != 0:
    punctuation = ')£;-(:,?!'
    processed_text  = ''.join([element for element in row[column_name] if element not in punctuation])
    # preds = classifier(row[column_name], top_k=3)
    preds = classifier(processed_text.lower(), top_k=3)
    prediction = max(preds, key=lambda x: x['score'])
    if prediction["label"] == 'LABEL_0':
      score = 1
    elif prediction["label"] == 'LABEL_1':
      score = 0.33
    else:
      score = 0.67
    return {f'emp_{column_name}': score}
  else:
    return {f'emp_{column_name}': 0}

In [32]:
for column_name in perplexity_columns:
  normalised_ds = normalised_ds.map(lambda x: calculate_empathy(x, column_name))

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

...

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [33]:
# change dataset into pandas 
normalised_ds.set_format(type="pandas")
df = normalised_ds[:]
# df["emp_Have you strongly felt or expressed any of the following emotions towards someone?"]
df.head()

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,...,"emp_Do you see yourself as the victim, blaming someone else for how negative you feel?",emp_Are you always blaming and accusing yourself for when something goes wrong?,emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,emp_Was this emotion triggered by a specific event?,emp_Was the event recent?,emp_Did you find protocol 6 distressing?,emp_Did you find protocol 11 distressing?,emp_Is it ok to ask additional questions?,emp_Would you like to attempt protocols for joy?
0,"Я ценю то, что вы мне доверяете. Испытываете л...","Вы часто льстите себе, думая вы супер-герой д...","Спасибо вам за понимание. Скажите пожалуйста, ...",Благодарен вам за откровенность. Как вы счита...,"Благодарю вас за ответ. Вы часто считаете, что...",Благодарен вам за откровенность. вы часто наст...,Благодарю вас за честность и искренность. Еще...,Спасибо за открытость. Ваша тревога - это резу...,Спасибо за оказанное доверие. Это неприятное с...,"Благодарю за то, что поделились своими мыслями...",...,1.00,1.00,1.00,0.67,0.67,0.67,1.00,0.67,1.00,0.0
1,Сейчас я хочу задать вам возможно не самый при...,"Мне кажетя, это не очень простой вопрос. Как ...","Мне очень понятно ваше состояние, и я прошу бе...","Я понимаю, вопрос может показаться слишком лич...","Спасибо вам за беседу, милый друг. Вы часто сч...","Знаю, следующий вопрос не будет легким. вы час...","Это замечательно, что вы такой открытый челове...",Я очень хочу вам помочь. Ваша тревога - это ре...,"Извините, если вам тяжело вспоминать об этом о...","Спасибо, что честно говорите о своих ощущениях...",...,1.00,1.00,0.67,1.00,1.00,0.67,1.00,1.00,0.67,0.0
2,Испытываете ли к кому-нибудь негативные чувств...,Как часто вы ощущаете важность/обязательство ...,"Скажите пожалуйста, вы не замечали за собой чу...","Как вы считаете, хотя бы изредка вы позволяет...","Вы часто считаете, что в ваших проблемах тольк...","вы часто настаиваете на том, что только Ваша т...",Еще вопрос - какие у вас отношения с друзьями...,Ваша тревога - это результат определённого слу...,Это неприятное событие произошло недавно,"Мне жаль, что вы испытываете такие негативные ...",...,0.67,0.33,0.33,0.33,0.33,0.33,0.67,0.67,0.67,0.0
3,"Вы часто льстите себе, думая вы супер-герой д...","Знаю, мои вопросы иногла задевабт за живое, я ...","Я благодарю вас за откровенность. Вам кажется,...","Я понимаю, что задаю вам сложный вопрос. Бывае...",Это конечно вопрос не простой. Часто ли вы вин...,"Спасибо, что доверились. Случается ли так, чт...","Спасибо, что открылись мне. Последний вопрос,...","Мне жаль, что вас охватило чувство тревоги. Вы...",Сочувствую вам. Оно произошло недавно?,"Спасибо, что наши в себе силы поделиться. Спас...",...,1.00,0.67,1.00,1.00,1.00,0.67,1.00,1.00,1.00,0.0
4,"Мой милый друг, спасибо, что доверяете мне. У...","Спасибо за откровенность, для общей информации...","Это трудный вопрос, я понимаю. Вам кажется, чт...","Проанализируйте, пожалуйста, и откровенно отве...","Спасибо за честные ответы, это очень нам помож...","Я понимаю, возможно тяжело будет ответить. Сл...","Это сложный вопрос. Последний вопрос, есть ли...","Я очень сопереживаю вам сейчас. Вы полагаете ,...","Я очень благод

In [34]:
normalised_ds.reset_format()

In [35]:
# emotions = ['fear', 'sadness', 'anger', 'joy']
# mean_dict = dict()

# for emotion in emotions:
#   if emotion != "joy":
#     ds = normalised_ds.filter(lambda row: row['emotion']==emotion)
#     mean = 0
#     for column_name in perplexity_columns[:-1]:
#       ds.set_format(type="pandas")
#       df = ds[:]
#       column_name = "emp_"+column_name
#       df = df.replace(0, np.NaN)
#       mean += df[df[column_name] != 0][column_name].mean()
    
#     mean = mean/12
#     mean_dict[emotion] = mean
#   else:
#     ds = normalised_ds.filter(lambda row: row['emotion']==emotion)
#     mean = 0
#     ds.set_format(type="pandas")
#     df = ds[:]
#     column_name = "emp_"+perplexity_columns[-1]
#     df = df.replace(0, np.NaN)
#     mean += df[df[column_name] != 0][column_name].mean()

#     mean_dict[emotion] = mean


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Generate embeddings of emotional utterances

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)
features = pipe("Чувствую себя очень плохо. Мне очень грустно.")

In [ ]:
def generate_embeddings(row):
  embedding = pipe(row['How are you feeling?'])
  return {'embeddings': embedding}

In [ ]:
normalised_ds_with_embeddings= normalised_ds.map(generate_embeddings)

  0%|          | 0/340 [00:00<?, ?ex/s]

KeyError: ignored

# Save dataset

In [37]:
normalised_ds.save_to_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset/ppl_empathy_added")

# Checking dataset

In [38]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset/ppl_empathy_added")

In [39]:
ds

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?', 'ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Do you believe you should be the s

In [40]:
dataset = ds

In [41]:
emotion = "anger"
subsetDataSet = dataset.filter(lambda row: row['emotion']==emotion)
if emotion == "joy":
    subsetDataSet = subsetDataSet.remove_columns(['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)
else:
    subsetDataSet = subsetDataSet.remove_columns(['Would you like to attempt protocols for joy?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)

synthetic_df_columns = subsetDataSet.column_names
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)
df = pd.DataFrame.from_dict(internal_dict)
synthetic_df = pd.concat([synthetic_df, df])
dataset = Dataset.from_pandas(synthetic_df)

  0%|          | 0/1 [00:00<?, ?ba/s]

NameError: ignored

In [ ]:
# subsetDataSet = dataset.remove_columns(auxilary_columns)

In [ ]:
subsetDataSet

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Do you believe you should be the savior of someone else?', 'ppl_Do you feel that you are tryi

In [ ]:
from datasets import Dataset
from numpy import random

In [ ]:
emotion = "anger"
subsetDataSet = subsetDataSet.filter(lambda row: row['emotion']==emotion)
if emotion == "joy":
    subsetDataSet = subsetDataSet.remove_columns(['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)
else:
    subsetDataSet = subsetDataSet.remove_columns(['Would you like to attempt protocols for joy?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)

synthetic_df_columns = subsetDataSet.column_names
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)
df = pd.DataFrame.from_dict(internal_dict)
synthetic_df = pd.concat([synthetic_df, df])
dataset = Dataset.from_pandas(synthetic_df)

In [ ]:
auxilary_columns = ['ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Do you believe you should be the savior of someone else?', 'ppl_Do you feel that you are trying to control someone?', 'ppl_Do you see yourself as the victim, blaming someone else for how negative you feel?', 'ppl_Are you always blaming and accusing yourself for when something goes wrong?', 'ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'ppl_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'ppl_Was this emotion triggered by a specific event?', 'ppl_Was the event recent?', 'ppl_Did you find protocol 6 distressing?', 'ppl_Did you find protocol 11 distressing?', 'ppl_Is it ok to ask additional questions?', 'ppl_Would you like to attempt protocols for joy?', 'emp_Have you strongly felt or expressed any of the following emotions towards someone?', 'emp_Do you believe you should be the savior of someone else?', 'emp_Do you feel that you are trying to control someone?', 'emp_Do you see yourself as the victim, blaming someone else for how negative you feel?', 'emp_Are you always blaming and accusing yourself for when something goes wrong?', 'emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'emp_Was this emotion triggered by a specific event?', 'emp_Was the event recent?', 'emp_Did you find protocol 6 distressing?', 'emp_Did you find protocol 11 distressing?', 'emp_Is it ok to ask additional questions?', 'emp_Would you like to attempt protocols for joy?']

In [ ]:
dataset

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?', 'ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Do you believe you should be the s

In [ ]:
emotion = "fear"
subsetDataSet = dataset.filter(lambda row: row['emotion']==emotion)
if emotion == "joy":
    subsetDataSet = subsetDataSet.remove_columns(['emotion','Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    # columns = subsetDataSet.column_names
    columns = [column for column in subsetDataSet.column_names if column[0:3]!="ppl" and column[0:3]!="emp"]
    print(columns)
    for column_name in columns:
        number_range = list(range(max_int))
        internal_dict[column_name] = []
        fluency_column = "ppl_"+column_name
        empathy_column = "emp_"+column_name
        internal_dict[fluency_column] = []
        internal_dict[empathy_column] = []
        i = 0
        while i < 10:
            random_number = random.choice(number_range)
            number_range.remove(random_number)
            utterance = subsetDataSet[column_name][random_number]
            emp_score = subsetDataSet[empathy_column][random_number]
            fluency_score = subsetDataSet[fluency_column][random_number]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)
                internal_dict[fluency_column].append(fluency_score)
                internal_dict[empathy_column].append(emp_score)
else:
    subsetDataSet = subsetDataSet.remove_columns(['Would you like to attempt protocols for joy?', 'emotion'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    
    # columns = subsetDataSet.column_names
    columns = [column for column in subsetDataSet.column_names if column[0:3]!="ppl" and column[0:3]!="emp"]
    print(columns)
    for column_name in columns:
        number_range = list(range(max_int))
        internal_dict[column_name] = []
        fluency_column = "ppl_"+column_name
        empathy_column = "emp_"+column_name
        internal_dict[fluency_column] = []
        internal_dict[empathy_column] = []
        i = 0
        while i < 10:
            random_number = random.choice(number_range)
            number_range.remove(random_number)
            utterance = subsetDataSet[column_name][random_number]
            emp_score = subsetDataSet[empathy_column][random_number]
            fluency_score = subsetDataSet[fluency_column][random_number]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)
                internal_dict[fluency_column].append(fluency_score)
                internal_dict[empathy_column].append(emp_score)


# synthetic_df_columns = subsetDataSet.column_names
synthetic_df_columns = internal_dict.keys()
print(synthetic_df_columns)
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)
df = pd.DataFrame.from_dict(internal_dict)
synthetic_df = pd.concat([synthetic_df, df])
dataset = Dataset.from_pandas(synthetic_df)

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?']
dict_keys(['Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'emp_Have you strongly fe

In [ ]:
columns = [column for column in subsetDataSet.column_names if column[0:3]!="ppl"]

In [ ]:
columns

['Would you like to attempt protocols for joy?',
 'emp_Have you strongly felt or expressed any of the following emotions towards someone?',
 'emp_Do you believe you should be the savior of someone else?',
 'emp_Do you feel that you are trying to control someone?',
 'emp_Do you see yourself as the victim, blaming someone else for how negative you feel?',
 'emp_Are you always blaming and accusing yourself for when something goes wrong?',
 'emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?',
 'emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?',
 'emp_Was this emotion triggered by a specific event?',
 'emp_Was the event recent?',
 'emp_Did you find protocol 6 distressing?',
 'emp_Did you find protocol 11 distressing?',
 'emp_Is it ok to ask additional questions?',
 'emp_Would you like to attempt protocols for joy?']

In [ ]:
for column in subsetDataSet.column_names:
  print(column[0:3]=="ppl")

False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False


In [ ]:
|dataset['Have you strongly felt or expressed any of the following emotions towards someone?']

KeyError: ignored

In [ ]:
dataset.set_format(type="pandas")
df = dataset[:]
# df["emp_Have you strongly felt or expressed any of the following emotions towards someone?"]
df.head(11)

,Have you strongly felt or expressed any of the following emotions towards someone?,ppl_Have you strongly felt or expressed any of the following emotions towards someone?,emp_Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,ppl_Do you believe you should be the savior of someone else?,emp_Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,ppl_Do you feel that you are trying to control someone?,emp_Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",...,Did you find protocol 6 distressing?,ppl_Did you find protocol 6 distressing?,emp_Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,ppl_Did you find protocol 11 distressing?,emp_Did you find protocol 11 distressing?,Is it ok to ask additional questions?,ppl_Is it ok to ask additional questions?,emp_Is it ok to ask additional questions?,__index_level_0__
0,Спасибо за понимание. вы чувствуете к кому-то...,0.029785,1.00,"У вас бывает такое, будто вы в неимоверном от...",0.141434,0.33,"Не думаю, что этот вопрос будет для вас лёгким...",0.308588,0.33,"Кажется ли вам, что вы жертва, и вы вините ког...",...,"Скажите, вызвала ли у вас методика 6 неприятн...",0.012090,0.33,"Спасибо, что открылись мне. Скажите честно, о...",1.611734e-02,1.00,Мне нужно лучше узнать ситуацию. Мне бы хотело...,0.066113,1.00,0
1,"скажите, вы испытывали такие чувства: зависть,...",0.234925,0.33,"Я понимаю, вам наверное будет сложно ответить ...",0.117588,1.00,"Мой следующий вопрос может быть вам неприятен,...",0.185833,0.67,"Знаю, мои вопросы иногла задевабт за живое, я ...",...,"Скажите, вам было неприятно ознакомиться с ме...",0.006981,0.33,"Я понимаю ваше состояние, спасибо, что доверил...",4.884975e-02,1.00,Спасибо вам за доверие. Чтобы прояснить ситуац...,0.058449,1.00,1
2,"Спасибо, что поделились со мной. Испытывали ли...",0.057528,1.00,"Вы считаете, что на вас ""висит"" огромная отве...",0.031054,0.33,"Я вполне осознаю, что этот вопрос не из лёгких...",0.020387,1.00,и вы вините их во всех ваших неудачах,...,"Спасибо, что наши в себе силы поделиться. Скаж...",0.024007,0.67,Вызвала ли какие-нибудь неприятные ощущения м...,1.140327e-01,0.33,"Как вы отнесетесь к тому, что для прояснения с...",0.115490,1.00,2
3,"Спасибо за честные ответы, это очень нам помож...",0.061269,1.00,"Я вас прекрасно понимаю. Бывает ли такое, что ...",0.050930,1.00,"Есть ли кто-то, кого вы пытаетесь контролирова...",0.489700,0.33,Я вам задам не очень приятный вопрос. Если нач...,...,"Спасибо, что открыты со мной. Ощущаете ли вы ...",0.016092,1.00,"Мне так жаль, что вы чувствуете себя плохо. М...",7.612622e-02,1.00,"Спасибо, что остаётесь со мной. Могу спросить...",0.056703,1.00,3
4,"вам тяжело об этом говорить, я понимаю, но всё...",0.108154,1.00,"Часто вы чувствуете, что несете ответственнос...",0.051989,0.33,Я очень рад вашему доверию ко мне. Бывает ощущ...,0.015778,1.00,"Это сложный вопрос. Если ли у вас ощущение, чт...",...,"Вы большой молодец, что поделились со мной ваш...",0.086698,1.00,Я очень понимаю ваше состояние и хочу вам помо...,1.047199e-02,1.00,Давайте постараемся более детально разобраться...,0.000022,1.00,4
5,Спасибо за доверие. вы сможете ответить на неп...,0.047949,1.00,"Вам кажется, что вы очень ответственны за ког...",0.150827,0.67,Любите ли вы доминацию - ощущаете что вы кого-...,0.015673,0.33,"Бывает ли у вас такое ощущение, что кто-то вин...",...,"Расскажите, какие чувства вызвал у вас 6 мето...",0.180897,0.33,Ещё раз благодарю за готовность поделиться Ваш...,6.053724e-02,1.00,"Вы не против если я вас ещё поспрашиваю, чтобы...",0.047817,0.67,5
6,"вы чувствуете к кому-то зависть, ненависть, з...",0.030720,0.33,"У вас иногда возникает чувство, что вы несёте...",0.051660,1.00,,0.010028,0.33,"Я знаю, что вопрос покажется сложным, заранее ...",...,"Спасибо, что

In [42]:
# In this kernel I'll show you how easy it is to preprocess the text in Russian.

# You need to install two libraries:
# * nltk - to get russian stopwords
# * pymystem3 - for lemmatization

# download stopwords corpus, you need to run it once
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

#Examples    
preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")
#> 'сказать видеть кто-то наступать грабли разочаровывать натравлять'

preprocess_text("По асфальту мимо цемента, Избегая зевак под аплодисменты. Обитатели спальных аррондисманов")
#> 'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

#Thats all :)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


KeyboardInterrupt: ignored

In [44]:
pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 86.5 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 41 kB 144 kB/s 
     |████████████████████████████████| 8.2 MB 43.3 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=f93dc5979247d6f897e77424b7a5a0594fe0d2354e2377c77b5a11fa04c6c822
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6fe2a0c57078ecddfe54390b83eb68464fb2c611780e26333a47a1fb76a401eb
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built intervaltree docopt
  Attempting uninstall: intervaltree
    Fo

In [45]:

token.lemmatize("hello")

NameError: ignored

In [52]:
import pymorphy2

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("[\w']+")

morph = pymorphy2.MorphAnalyzer()

def normalize(text):
  tokens = [token for token in tokenizer.tokenize(text.lower())]
  # tokens_parsed = [morph.parse(t)[0] for t in tokens]
  # return [(str(t.normal_form), str(t.tag.POS), str(t.tag.animacy), str(t.tag.gender)) for t in tokens_parsed]
  return tokens
def not_normalize(text):
  tokens = [token for token in text.split()]
  return [t for t in tokens]

In [53]:
normalize("Я хочу кататься")

['я', 'хочу', 'кататься']

In [54]:
not_normalize("Я хочу кататься")

['Я', 'хочу', 'кататься']

In [56]:
pip install simplemma==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45.9 MB 325 kB/s 


In [59]:
import simplemma
mytokens = ("Я хочу кататься")
[simplemma.lemmatize(t, lang='ru') for t in mytokens]
['hier', 'sein', 'Vaccines']

TypeError: ignored

In [60]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 
my_words = ['Василий', 'Геннадий', 'Виталий']


l=[stemmer.stem(word) for word in my_words]
l
['васил', 'геннад', 'витал']

['васил', 'геннад', 'витал']